# Getting 2018 for Imputation 

## loading necesary functions - bleh urls broken rn 

In [1]:
# %load ../api_access_snippet.py
#import library
import gspread 
#Service client credential from oauth2client
from oauth2client.service_account import ServiceAccountCredentials
# Print nicely
import pprint
#Create scope
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
#create some credential using that scope and content of startup_funding.json
creds = ServiceAccountCredentials.from_json_keyfile_name('../quickstart/g_sheet_creds.json',scope)
#create gspread authorize using that credential
client = gspread.authorize(creds)
my_email = 'matthewjchristy66@gmail.com'

def read_file(sheet_name):
    out = client.open(sheet_name).sheet1
    out = out.get_all_values()
    out = pd.DataFrame(out, columns = out.pop(0))
    return(out)

In [2]:
from requests import get 
import lxml.html as lh
import pandas as pd
from bs4 import BeautifulSoup 

In [3]:
# %load write_function.py
def writer(data, sheet_name, share_email):
    #Grabbing Parameters for looping 
    n_rows = data.shape[0]
    n_cols = data.shape[1]
    
    #creating sheets
     #Now will can access our google sheets we call client.open on StartupName
    sheet = client.create(sheet_name) 
    sheet.share(share_email,  perm_type='user', role='writer') #sharing my email 
    
    #getting cell list to batch update
    import string
    end_col = string.ascii_uppercase[n_cols - 1]
    end_row = n_rows + 1
    
    sheet_range = 'A1:'+ end_col + str(end_row)
    
    #turning df to one long list 
    df_as_list = data.stack().tolist()
    df_as_list = data.columns.tolist() + df_as_list
    
    #getting the target sheet 
    ws = sheet.get_worksheet(0)
    cell_list = ws.range(sheet_range)
    
    #writing df list to cell range list 
    for i in range(0, len(cell_list)):
        cell_list[i].value = df_as_list[i]
        
    #batch updating 
    ws.update_cells(cell_list)

In [4]:
# %load scraping_functions.py
def get_url(url):
    #grabbing the HTML and getting text 
    my_header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.97 Safari/537.36'}
    fantasy_page = get(url, headers = my_header)
    
    return(fantasy_page)
    
def get_headers(html):
    
    doc = lh.fromstring(html.content)
    
    #getting column names 
    title = doc.xpath('//tr//th')
    
    colnames = []

    n = len(title)

    for i in range(0, n):
        name = title[i].text_content()
        colnames.append(name)
    
    return(colnames)    
    
    
def get_table_data(html, colnames):
     
    #parsing table elements in the HTML inside the pattern "//tr" --> this is a table element 
    doc = lh.fromstring(html.content)
    table_elements = doc.xpath('//tr')
    
    #creating an empty array to insert the table elements 
    cols = []

    for j in range(0, len(colnames)):
        name = colnames[j] #getting the column name from the HTML table 
        #print('%d:"%s"'% (i, name))
        cols.append((name, []))

   #Since out first row is the header, data is stored on the second row onwards

    for j in range(1,len(table_elements)):
        #T is our j'th row
        T=table_elements[j]

        #If row is not of size 24, the //tr data is not from our table 
        if len(T)!=len(colnames):
            break

        #i is the index of our column
        i=0

        #Iterate through each element of the row
        for t in T.iterchildren():
            data=t.text_content() 

            #Append the data to the empty list of the i'th column
            cols[i][1].append(data)
            #Increment i for the next column
            i+=1
    return(cols)
    
def create_df(cols):
    #creating a dictionary for the columns in the parsed table 
    Dict={title:column for (title,column) in cols}

    df=pd.DataFrame(Dict)
    return(df)
    
def clean_df(df, page_url, select_vars):
    #data cleaning 
    escapes = ''.join([chr(char) for char in range(1, 32)])
    translator = str.maketrans('', '', escapes)
    df.columns = df.columns.str.translate(translator)
    
    #fixing escape charaters
    fixed = select_vars # ['Player', 'Team', 'Yds/G', 'TD', 'FUM']
    for i in fixed:
        df.loc[:, i] = df.loc[:, i].astype(str).str.translate(translator)
   
    #import re 
    #week = re.search('week=(.*)&season', urls[0]).group(1)
    #df.insert(1, 'Week', week)
    
    #casting to strings 
    df = df.astype(str)
       
    #returning the df
    return(df)

#### make urls to get data 

In [5]:
stat_cat = ['PASSING', 'RUSHING', 'RECEIVING']
pages = [x for x in range(1, 4)]

part_1 = 'http://www.nfl.com/stats/categorystats?tabSeq=0&season=2018&seasonType=REG&d-447263-n=1&d-447263-o=2&statisticCategory='
part_2 = '&conference=null&d-447263-p='
#part_3 = '&d-447263-s=RUSHING_YARDS'

urls = []

for i in stat_cat:
    for j in pages:
        full_url = part_1 + i + part_2 + str(j) #+ part_3
        urls.append(full_url)


In [6]:
passing = pd.DataFrame()
rec = pd.DataFrame()
rush = pd.DataFrame()

In [9]:
for i in range(0, len(urls)): 
    source_html = get_url(urls[i])
    colnames = get_headers(source_html)
    table_data = get_table_data(source_html, colnames)
    raw_df = create_df(table_data)
        
    if 'PASSING' in urls[i]: 
        df = clean_df(raw_df, urls[i], select_vars = ['Player', 'Team', 'Yds/G', 'TD', 'Int','Rate'])
        df = df[['Player', 'Team', 'Yds/G', 'TD', 'Int','Rate']]
        passing = pd.concat([passing, df])
    
    elif 'RECEIVING' in urls[i]: 
        df = clean_df(raw_df, urls[i], select_vars = ['Player', 'Team', 'Yds/G', 'TD', 'FUM'])
        df = df[['Player', 'Team', 'Yds/G', 'TD', 'FUM']]
        receiving = pd.concat([receiving, df])
        
    else: 
        df = clean_df(raw_df, urls[i], select_vars = ['Player', 'Team', 'Yds/G', 'TD', 'FUM'])
        df = df[['Player', 'Team', 'Yds/G', 'TD', 'FUM']]
        rushing = pd.concat([rushing, df])
    
    print(i)

0
1
2
3
4
5
6
7
8


In [10]:
print('Passing', passing.shape)

print('rec', receiving.shape)

print('rush', rushing.shape)

Passing (212, 6)
rec (200, 5)
rush (200, 5)


## Score the Data

In [11]:
passing[['Yds/G', 'TD', 'Int','Rate']] = passing[['Yds/G', 'TD', 'Int','Rate']].astype(float)
passing['2018_ppg'] = (passing['Yds/G'] * .04) + ((4 * passing['TD'])/16) - (passing['Int']/16)

In [12]:
rushing[['Yds/G', 'TD', 'FUM']] = rushing[['Yds/G', 'TD', 'FUM']].astype(float)
rushing['2018_ppg'] = (rushing['Yds/G'] * .1) + (rushing['TD'] * 6/16) - (rushing['FUM']/16)

In [13]:
receiving[['Yds/G', 'TD', 'FUM']] = receiving[['Yds/G', 'TD', 'FUM']].astype(float)
receiving['2018_ppg'] = (receiving['Yds/G'] * .1) + (receiving['TD'] * 6/16) - (receiving['FUM']/16)

## Writing the data 

In [14]:
writer(data = passing, sheet_name = '2018_passing_recap', share_email = my_email)

In [15]:
writer(data = rushing, sheet_name = '2018_rushing_recap', share_email = my_email)

In [16]:
writer(data = receiving, sheet_name = '2018_receiving_recap', share_email = my_email)